In [1]:
import os
from dotenv import load_dotenv, find_dotenv
from langchain_groq import ChatGroq

_ = load_dotenv(find_dotenv())

# api for groq
GROQ_API = os.getenv("GROQ_API")
# api for openai
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

chat_groq_model = ChatGroq(
    model='llama-3.1-70b-versatile',
    api_key = GROQ_API,
    temperature= 0.7,
    max_retries=3,
)

In [12]:
from langchain_community.document_loaders import TextLoader,PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma
# load data

text_documents = TextLoader('D:/Data_Science_Study/Course_Pracitse_Code/GenerativeAi/Langchain_101/data/state_of_the_union.txt',autodetect_encoding = True).load()

# split documents
text_splitter = CharacterTextSplitter(chunk_size = 1000,
                                      chunk_overlap = 200)

chunk_of_text = text_splitter.split_documents(text_documents)

In [14]:
vector_db = Chroma.from_documents(chunk_of_text,OpenAIEmbeddings())

In [15]:
question = "what did the president say about Jhon Lewis voting right act?"

response = vector_db.similarity_search(question)
print(response[0].page_content)

In state after state, new laws have been passed, not only to suppress the vote, but to subvert entire elections. 

We cannot let this happen. 

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


In [16]:
from langchain_community.vectorstores import FAISS

In [17]:
vector_db_F = FAISS.from_documents(chunk_of_text,OpenAIEmbeddings())

In [18]:
retriever_F = vector_db_F.as_retriever()

In [19]:
response = retriever_F.invoke("what did he say about ketanji brown jackson?")

In [20]:
response

[Document(id='9deea882-5b32-4147-9b2e-66ff1b20f5e4', metadata={'source': 'D:/Data_Science_Study/Course_Pracitse_Code/GenerativeAi/Langchain_101/data/state_of_the_union.txt'}, page_content='And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence. \n\nA former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since she’s been nominated, she’s received a broad range of support—from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. \n\nAnd if we are to advance liberty and justice, we need to secure the Border and fix the immigration system. \n\nWe can do both. At our border, we’ve installed new technology like cutting-edge scanners to better detect drug smuggling.  \n\nWe’ve set up joint patrols with Mexico and Guate

In [21]:
retriever_FK = vector_db_F.as_retriever(search_kwargs = {'k' : 1})

In [22]:
response = retriever_FK.invoke("what did he say about ketanji brown jackson?")

In [23]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough,RunnableParallel

In [25]:
retriever_a = vector_db_F.as_retriever()

In [26]:
retrieval = RunnableParallel(
    {"context" : retriever_a, "question": RunnablePassthrough()}
)